# AoC - Day 4

## Part 1 - Problem Statement

You arrive at the Venus fuel depot only to discover it's protected by a password. The Elves had written the password on a sticky note, but someone threw it out.

However, they do remember a few key facts about the password:

* It is a six-digit number.
* The value is within the range given in your puzzle input.
* Two adjacent digits are the same (like 22 in 122345).
* Going from left to right, the digits never decrease; they only ever increase or stay the same (like 111123 or 135679).


Other than the range rule, the following are true:

* 111111 meets these criteria (double 11, never decreases).
* 223450 does not meet these criteria (decreasing pair of digits 50).
* 123789 does not meet these criteria (no double).


How many different passwords within the range given in your puzzle input meet these criteria?

## Solution

In [1]:
def six_digits(number):
    return 100000 <= number < 1000000

In [3]:
def make_in_range(lower, upper):
    def in_range(number):
        return lower <= number <= upper
    
    return in_range

In [6]:
def listify_number(number):
    return [int(digit) for digit in str(number)]

In [14]:
def digits_match(digit_list):
    first_digit = digit_list[0]
    return all(digit == first_digit for digit in digit_list)

In [17]:
def adjacent_pairs_match(digit_list):
    return any(digits_match(pair) for pair in zip(digit_list[:-1], digit_list[1:]))

In [27]:
def non_decreasing(digit_list):
    return all(left - right <= 0 for left, right in zip(digit_list[:-1], digit_list[1:]))

In [38]:
def build_matcher(lower, upper):
    in_range = make_in_range(lower, upper)
    
    def matcher(number):
        if not six_digits(number) or not in_range(number):
            return False
        
        digit_list = listify_number(number)
        return adjacent_pairs_match(digit_list) and non_decreasing(digit_list)
    
    return matcher

In [30]:
def count_matches(lower, upper):
    meets_criteria = build_matcher(lower, upper)
    
    return len([True for number in range(lower, upper+1) if meets_criteria(number)])

## Tests

In [25]:
assert six_digits(100000)
assert six_digits(999999)
assert not six_digits(99999)
assert not six_digits(1000000)

In [24]:
assert make_in_range(10, 20)(10)
assert make_in_range(10, 20)(20)
assert not make_in_range(10, 20)(9)
assert not make_in_range(10, 20)(21)

In [7]:
assert listify_number(123456) == range(1, 7)

In [23]:
assert digits_match([1, 1])
assert not digits_match([1, 2])

In [22]:
assert adjacent_pairs_match([1] * 6)
assert adjacent_pairs_match(listify_number(122345))

In [28]:
assert non_decreasing(listify_number(111111))
assert non_decreasing(listify_number(122345))
assert not non_decreasing(listify_number(213456))

In [36]:
assert build_matcher(100000, 199000)(123345)
assert not build_matcher(100000, 199000)(123456)

In [45]:
assert count_matches(100000, 111111) == 1
assert count_matches(100000, 111112) == 2

## Solution

In [46]:
count_matches(245182, 790572)

1099

## Part 2 - Problem Statement

An Elf just remembered one more important detail: the two adjacent matching digits are not part of a larger group of matching digits.

Given this additional criterion, but still ignoring the range rule, the following are now true:

* 112233 meets these criteria because the digits never decrease and all repeated digits are exactly two digits long.
* 123444 no longer meets the criteria (the repeated 44 is part of a larger group of 444).
* 111122 meets the criteria (even though 1 is repeated more than twice, it still contains a double 22).


How many different passwords within the range given in your puzzle input meet all of the criteria?

## Solution

In [70]:
def build_runs_list(digit_list):

    last_pair_matched = False
    runs_list = []
    for left, right in zip(digit_list[:-1], digit_list[1:]):
        digits_match = left == right
        if not last_pair_matched and digits_match:
            runs_list.append([left, right])
            last_pair_matched = True
        elif last_pair_matched and digits_match:
            runs_list[-1].append(left)
        else:
            last_pair_matched = False
            
    return runs_list

In [75]:
def isolated_matching_pair(digit_list):
    
    runs_list = build_runs_list(digit_list)
    run_lengths = [len(run) for run in runs_list]
    
    return any(length == 2 for length in run_lengths)

In [78]:
def build_matcher_2(lower, upper):
    in_range = make_in_range(lower, upper)
    
    def matcher(number):
        if not six_digits(number) or not in_range(number):
            return False
        
        digit_list = listify_number(number)
        return isolated_matching_pair(digit_list) and non_decreasing(digit_list)
    
    return matcher

In [79]:
def count_matches_2(lower, upper):
    meets_criteria = build_matcher_2(lower, upper)
    
    return len([True for number in range(lower, upper+1) if meets_criteria(number)])

## Tests

In [74]:
assert build_runs_list(listify_number(112233)) == [[1, 1], [2, 2], [3, 3]]
assert build_runs_list(listify_number(123444)) == [[4, 4, 4]]
assert build_runs_list(listify_number(111122)) == [[1, 1, 1, 1], [2, 2]]

In [77]:
assert isolated_matching_pair(listify_number(112233))
assert not isolated_matching_pair(listify_number(123444))
assert isolated_matching_pair(listify_number(111122))

In [80]:
count_matches_2(245182, 790572)

710